# Relationalizing & Visualizing CLRS Data Structures with sPyTial

This notebook demonstrates how to visualize classic data structures from "Introduction to Algorithms" (CLRS) using sPyTial. We'll show how these structures are "relationalized" — transformed into atoms and relations — and then spatially visualized with meaningful layout constraints.

## Setup

In [2]:
import sys
from pathlib import Path

# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

# Import sPyTial components
from spytial import diagram, orientation, group, atomColor, attribute, hideField, cyclic, evaluate
from spytial.annotations import (
    annotate_orientation, annotate_group, annotate_atomColor, 
    annotate_attribute, annotate_cyclic, annotate_flag
)

# For more complex examples
import random
from typing import Optional, List, Dict, Any
import heapq

print("sPyTial setup complete!")
print("Ready to visualize CLRS data structures.")

sPyTial setup complete!
Ready to visualize CLRS data structures.


In [3]:
# Show version info
import spytial
print(f"sPyTial version: {getattr(spytial, '__version__', 'development')}")

sPyTial version: development


## Arrays & Dynamic Arrays

**Relationalization**: Array indices become relations `indexOf(i, elem)` connecting positions to values.

**Visualization**: Horizontal orientation left→right shows natural array ordering; values are labeled clearly.

**What to notice**: The linear flow makes array structure immediately apparent, and operations like insertion/deletion show their positional effects.

In [4]:
# Basic array demonstration
simple_array = [10, 25, 30, 15, 20]

## Why so complicated?
## Arrays are more than just a list of numbers.
## We want to show the indices, and we want to show the ordering of the numbers.
sa = orientation(selector='{x,y : idx[object][object] | @num:(x[idx[object]]) < @num:(y[idx[object]])}', directions=['directlyRight'])(simple_array)
sa = annotate_flag(sa,name = 'hideDisconnected')

diagram(sa)

In [30]:
# Array operations demo
dynamic_array = [1, 2, 3]
dynamic_array = orientation(selector='{x,y : idx[object][object] | @num:(x[idx[object]]) < @num:(y[idx[object]])}', directions=['directlyRight'])(dynamic_array)
dynamic_array = annotate_flag(dynamic_array,name = 'hideDisconnected')

print("Initial array:")
diagram(dynamic_array)

print("\nAfter appending 4:")
dynamic_array.append(4)  # Append operation
diagram(dynamic_array)

print("\nAfter inserting 99 at position 1:")
dynamic_array[1] = 99  # Insert operation
diagram(dynamic_array)

Initial array:



After appending 4:



After inserting 99 at position 1:


## Linked Lists (Singly & Doubly), Stacks, Queues, Deques

**Relationalization**: 
- Singly linked: Nodes as atoms, `next(x, y)` relations
- Doubly linked: Add `prev(x, y)` relations
- Stacks/Queues/Deques: Views over linked structures with head/tail markers

**Visualization**: Linear flow with directional arrows; optional grouping for special positions (head/tail).

**What to notice**: The pointer-following structure becomes visually clear, and stack/queue operations show how elements flow through the structure.

In [ ]:

@orientation(selector='{x,y : SinglyNode | x->y in next}', directions=['directlyRight'])
class SinglyNode:
    def __init__(self, data, next_node=None):
        self.data = data
        self.next = next_node
    
    def __repr__(self):
        return f"Node({self.data})"

@orientation(selector='{x, y : DoublyNode | x->y in next}', directions=['directlyRight'])
@orientation(selector='{x, y : DoublyNode | x->y in prev}', directions=['directlyLeft'])
class DoublyNode:
    def __init__(self, data, next_node=None, prev_node=None):
        self.data = data
        self.next = next_node
        self.prev = prev_node
    
    def __repr__(self):
        return f"DNode({self.data})"


Linked list node classes defined.


In [34]:
# Singly linked list demonstration
head = SinglyNode(1)
head.next = SinglyNode(2)
head.next.next = SinglyNode(3)

print("Singly linked list (1 -> 2 -> 3):")
diagram(head)

# Demonstrate insertion
new_head = SinglyNode(0)
new_head.next = head

print("\nAfter inserting 0 at head (0 -> 1 -> 2 -> 3):")
diagram(new_head)

Singly linked list (1 -> 2 -> 3):



After inserting 0 at head (0 -> 1 -> 2 -> 3):


In [35]:
# Doubly linked list demonstration
node1 = DoublyNode(10)
node2 = DoublyNode(20)
node3 = DoublyNode(30)

# Link them together
node1.next = node2
node2.prev = node1
node2.next = node3
node3.prev = node2

print("Doubly linked list (10 <-> 20 <-> 30):")
diagram(node1)

Doubly linked list (10 <-> 20 <-> 30):


In [37]:
# Stack and Queue as list views
stack_data = [1, 2, 3, 4]  # top is at the end
diagram(stack_data)

# Queue demonstration
queue_data = [1, 2, 3, 4]  # front=0, rear=3
annotate_orientation(queue_data, selector='items', directions=['horizontal'])
# Color first and last elements differently
annotate_atomColor([queue_data[0]], selector='self', value='green')  # front
annotate_atomColor([queue_data[-1]], selector='self', value='blue')  # rear

print("\nQueue visualization (front=green, rear=blue):")
diagram(queue_data)


Queue visualization (front=green, rear=blue):


## Heaps / Priority Queues (Binary Heap)

**Relationalization**: Tree structure with `left(parent, child)` and `right(parent, child)` relations; `keyOf(node, priority)` for values.

**Visualization**: Parent nodes appear above children; heap property becomes visually apparent through the tree layout.

**What to notice**: The tree structure makes the heap property easy to verify visually — parents are always above children with appropriate priority ordering.

In [38]:
# Binary heap node with tree layout
@orientation(selector='left', directions=['below', 'left'])
@orientation(selector='right', directions=['below', 'right'])
@attribute(field='priority')
class HeapNode:
    def __init__(self, priority, left=None, right=None):
        self.priority = priority
        self.left = left
        self.right = right
    
    def __repr__(self):
        return f"Heap({self.priority})"

print("Binary heap node class defined.")

Binary heap node class defined.


In [39]:
# Min-heap demonstration
root = HeapNode(1)  # minimum at root
root.left = HeapNode(3)
root.right = HeapNode(2)
root.left.left = HeapNode(7)
root.left.right = HeapNode(8)
root.right.left = HeapNode(4)

print("Min-heap visualization:")
diagram(root)

# Show insertion effect
# After inserting 0 (new minimum)
new_root = HeapNode(0)  # new minimum
new_root.left = HeapNode(1)  # old root moves down
new_root.right = HeapNode(2)
new_root.left.left = HeapNode(3)
new_root.left.right = HeapNode(8)
new_root.right.left = HeapNode(4)
new_root.left.left.left = HeapNode(7)

print("\nAfter inserting 0 (new minimum):")
diagram(new_root)

Min-heap visualization:



After inserting 0 (new minimum):


In [12]:
# Priority queue using Python's heapq
pq = [(1, 'urgent'), (3, 'normal'), (2, 'important'), (5, 'low')]
heapq.heapify(pq)

# Show as array with heap property
annotate_orientation(pq, selector='items', directions=['horizontal'])

print("Priority queue as array (heap property maintained):")
print(f"Data: {pq}")
diagram(pq)

# Extract minimum
min_item = heapq.heappop(pq)
print(f"\nExtracted minimum: {min_item}")
print(f"Remaining heap: {pq}")
diagram(pq)

Priority queue as array (heap property maintained):
Data: [(1, 'urgent'), (3, 'normal'), (2, 'important'), (5, 'low')]



Extracted minimum: (1, 'urgent')
Remaining heap: [(2, 'important'), (3, 'normal'), (5, 'low')]


## 8. Binary Search Trees + Red-Black Trees

**Relationalization**: Same tree structure as heaps but with BST ordering invariant; Red-Black trees add `color` attributes to nodes.

**Visualization**: Parent above children, left child positioned left, right child positioned right; BST property becomes visually verifiable.

**What to notice**: The spatial layout makes BST ordering immediately apparent — smaller values flow left, larger values flow right through the tree structure.

In [13]:
# Binary Search Tree node
@orientation(selector='left', directions=['below', 'left'])
@orientation(selector='right', directions=['below', 'right'])
@attribute(field='key')
class BSTNode:
    def __init__(self, key, left=None, right=None):
        self.key = key
        self.left = left
        self.right = right
    
    def __repr__(self):
        return f"BST({self.key})"

# Red-Black Tree node
@orientation(selector='left', directions=['below', 'left'])
@orientation(selector='right', directions=['below', 'right'])
@attribute(field='key')
@atomColor(selector='self', value='red')  # default red, will be overridden
class RBNode:
    def __init__(self, key, color='red', left=None, right=None):
        self.key = key
        self.color = color
        self.left = left
        self.right = right
    
    def __repr__(self):
        return f"RB({self.key},{self.color[0]})"

print("BST and Red-Black tree node classes defined.")

BST and Red-Black tree node classes defined.


In [14]:
# Binary Search Tree demonstration
bst_root = BSTNode(5)
bst_root.left = BSTNode(3)
bst_root.right = BSTNode(7)
bst_root.left.left = BSTNode(1)
bst_root.left.right = BSTNode(4)
bst_root.right.left = BSTNode(6)
bst_root.right.right = BSTNode(9)

print("Binary Search Tree:")
print("Notice: left subtree < root < right subtree")
diagram(bst_root)

# Show insertion
# Insert 2 (goes left of 3, right of 1)
bst_root.left.left.right = BSTNode(2)

print("\nAfter inserting 2:")
diagram(bst_root)

Binary Search Tree:
Notice: left subtree < root < right subtree



After inserting 2:


In [15]:
# Red-Black Tree demonstration
rb_root = RBNode(7, 'black')  # root is always black
rb_root.left = RBNode(3, 'red')
rb_root.right = RBNode(11, 'red')
rb_root.left.left = RBNode(1, 'black')
rb_root.left.right = RBNode(5, 'black')
rb_root.right.left = RBNode(9, 'black')
rb_root.right.right = RBNode(13, 'black')

# Apply colors to nodes
annotate_atomColor([rb_root], selector='self', value='black')
annotate_atomColor([rb_root.left, rb_root.right], selector='self', value='red')
annotate_atomColor([rb_root.left.left, rb_root.left.right, 
                   rb_root.right.left, rb_root.right.right], selector='self', value='black')

print("Red-Black Tree (colors show balancing):")
print("Black root, red children, black grandchildren")
diagram(rb_root)

Red-Black Tree (colors show balancing):
Black root, red children, black grandchildren


## 9. Hash Tables (Chaining & Open Addressing)

**Relationalization**: 
- Chaining: Buckets as groups with `inBucket(elem, bucket)` and `next` within chains
- Open Addressing: Array view with probe sequences shown as thin edges

**Visualization**: Bucket grouping makes collision handling clear; probe sequences show how open addressing resolves conflicts.

**What to notice**: Hash table structure becomes immediately apparent — you can see bucket distribution, collision patterns, and load factors visually.

In [16]:
# Hash table with chaining
class HashChainNode:
    def __init__(self, key, value, next_node=None):
        self.key = key
        self.value = value
        self.next = next_node
    
    def __repr__(self):
        return f"({self.key}: {self.value})"

@orientation(selector='next', directions=['right'])
class ChainedHashNode(HashChainNode):
    pass

class HashTable:
    def __init__(self, size=7):
        self.size = size
        self.buckets = [None] * size
    
    def hash(self, key):
        return hash(key) % self.size
    
    def insert(self, key, value):
        index = self.hash(key)
        new_node = ChainedHashNode(key, value)
        if self.buckets[index] is None:
            self.buckets[index] = new_node
        else:
            new_node.next = self.buckets[index]
            self.buckets[index] = new_node
    
    def __repr__(self):
        return f"HashTable(size={self.size})"

print("Hash table classes defined.")

Hash table classes defined.


In [17]:
# Hash table with chaining demonstration
ht = HashTable(5)
ht.insert("apple", 1)
ht.insert("banana", 2)
ht.insert("cherry", 3)
ht.insert("date", 4)  # might collide

# Apply grouping to show buckets
annotate_orientation(ht.buckets, selector='items', directions=['vertical'])
for i, bucket in enumerate(ht.buckets):
    if bucket:
        annotate_group([bucket], field=f'bucket_{i}', groupOn=0, addToGroup=1)

print("Hash table with chaining:")
print("Each bucket can contain a chain of colliding elements")
diagram(ht.buckets)

# Show collision by adding more items
ht.insert("elderberry", 5)  # force collision
print("\nAfter adding more items (collisions create chains):")
diagram(ht.buckets)

Hash table with chaining:
Each bucket can contain a chain of colliding elements



After adding more items (collisions create chains):


In [18]:
# Open addressing hash table simulation
class OpenAddressTable:
    def __init__(self, size=7):
        self.size = size
        self.table = [None] * size
        self.deleted = [False] * size
    
    def hash(self, key):
        return hash(key) % self.size
    
    def insert(self, key, value):
        index = self.hash(key)
        while self.table[index] is not None and not self.deleted[index]:
            index = (index + 1) % self.size  # linear probing
        self.table[index] = (key, value)
        self.deleted[index] = False

# Demo open addressing
oa_table = OpenAddressTable(7)
oa_table.insert("x", 1)
oa_table.insert("y", 2)
oa_table.insert("z", 3)

# Show as array with horizontal flow
annotate_orientation(oa_table.table, selector='items', directions=['horizontal'])

print("Open addressing hash table:")
print("Linear probing resolves collisions by finding next empty slot")
print(f"Table contents: {oa_table.table}")
diagram(oa_table.table)

Open addressing hash table:
Linear probing resolves collisions by finding next empty slot
Table contents: [('y', 2), ('x', 1), ('z', 3), None, None, None, None]


## 10. Disjoint Set Union (Union-Find)

**Relationalization**: Forest structure with `parent(child, parent)` relations; `rank` or `size` attributes for optimization.

**Visualization**: Trees show connected components; roots are highlighted; union operations create visual merging of components.

**What to notice**: The forest structure makes connected components immediately visible — each tree represents one disjoint set, with the root as the representative.

In [19]:
# Union-Find / Disjoint Set Union
@orientation(selector='parent', directions=['above'])
@attribute(field='rank')
class UFNode:
    def __init__(self, value, parent=None, rank=0):
        self.value = value
        self.parent = parent if parent else self  # root points to itself
        self.rank = rank
    
    def __repr__(self):
        return f"UF({self.value})"

class UnionFind:
    def __init__(self, elements):
        self.nodes = {elem: UFNode(elem) for elem in elements}
    
    def find(self, x):
        node = self.nodes[x]
        if node.parent != node:
            node.parent = self.find(node.parent.value)  # path compression
        return node.parent
    
    def union(self, x, y):
        root_x = self.find(x)
        root_y = self.find(y)
        
        if root_x == root_y:
            return
        
        # Union by rank
        if root_x.rank < root_y.rank:
            root_x.parent = root_y
        elif root_x.rank > root_y.rank:
            root_y.parent = root_x
        else:
            root_y.parent = root_x
            root_x.rank += 1

print("Union-Find data structure defined.")

Union-Find data structure defined.


In [20]:
# Union-Find demonstration
uf = UnionFind([1, 2, 3, 4, 5])

# Initially all elements are separate (each is its own root)
print("Initial state - each element is its own set:")
roots = [node for node in uf.nodes.values() if node.parent == node]
annotate_atomColor(roots, selector='self', value='green')  # highlight roots
diagram(list(uf.nodes.values()))

# Perform some unions
uf.union(1, 2)  # connect 1 and 2
uf.union(3, 4)  # connect 3 and 4

print("\nAfter union(1,2) and union(3,4):")
print("Two connected components: {1,2} and {3,4}, plus singleton {5}")
roots = [node for node in uf.nodes.values() if node.parent == node]
annotate_atomColor(roots, selector='self', value='green')
diagram(list(uf.nodes.values()))

# Union the components
uf.union(2, 3)  # connect the two components

print("\nAfter union(2,3) - merging components:")
print("Now we have {1,2,3,4} and {5}")
roots = [node for node in uf.nodes.values() if node.parent == node]
annotate_atomColor(roots, selector='self', value='green')
diagram(list(uf.nodes.values()))

Initial state - each element is its own set:



After union(1,2) and union(3,4):
Two connected components: {1,2} and {3,4}, plus singleton {5}



After union(2,3) - merging components:
Now we have {1,2,3,4} and {5}


## 11. Graphs (Adjacency List/Matrix)

**Relationalization**: `adj(u, v)` relations for adjacency; optional `weight(edge, w)` for weighted graphs.

**Visualization**: Cyclic layout or layered orientation; edge attributes show weights; BFS/DFS trees can be overlaid as additional relations.

**What to notice**: Graph connectivity becomes immediately apparent; different layout strategies (cyclic, layered) emphasize different aspects of the graph structure.

In [21]:
# Graph representation classes
@attribute(field='value')
class GraphVertex:
    def __init__(self, value, neighbors=None):
        self.value = value
        self.neighbors = neighbors or []
    
    def add_neighbor(self, neighbor, weight=1):
        self.neighbors.append({'vertex': neighbor, 'weight': weight})
    
    def __repr__(self):
        return f"V({self.value})"

class WeightedEdge:
    def __init__(self, from_vertex, to_vertex, weight):
        self.from_vertex = from_vertex
        self.to_vertex = to_vertex
        self.weight = weight
    
    def __repr__(self):
        return f"Edge({self.from_vertex.value}->{self.to_vertex.value}, w={self.weight})"

print("Graph classes defined.")

Graph classes defined.


In [22]:
# Simple graph demonstration
# Create vertices
vertices = {i: GraphVertex(i) for i in range(5)}

# Add edges to create a small graph
vertices[0].add_neighbor(vertices[1], 2)
vertices[0].add_neighbor(vertices[2], 3)
vertices[1].add_neighbor(vertices[3], 1)
vertices[2].add_neighbor(vertices[3], 4)
vertices[3].add_neighbor(vertices[4], 2)

# Apply cyclic layout for graph
vertex_list = list(vertices.values())
annotate_cyclic(vertex_list, selector='root', direction='clockwise')

print("Weighted directed graph:")
print("Vertices arranged in circular layout")
diagram(vertex_list)

Weighted directed graph:
Vertices arranged in circular layout


In [23]:
# Adjacency matrix representation
adj_matrix = [
    [0, 2, 3, 0, 0],  # vertex 0 connections
    [0, 0, 0, 1, 0],  # vertex 1 connections
    [0, 0, 0, 4, 0],  # vertex 2 connections
    [0, 0, 0, 0, 2],  # vertex 3 connections
    [0, 0, 0, 0, 0]   # vertex 4 connections
]

# Show matrix structure
for i, row in enumerate(adj_matrix):
    annotate_orientation(row, selector='items', directions=['horizontal'])

annotate_orientation(adj_matrix, selector='items', directions=['vertical'])

print("Adjacency matrix representation:")
print("Same graph as above, shown as 5x5 matrix")
print("Non-zero entries indicate edges and their weights")
diagram(adj_matrix)

Adjacency matrix representation:
Same graph as above, shown as 5x5 matrix
Non-zero entries indicate edges and their weights


In [24]:
# BFS tree overlay demonstration
# Simple BFS tree from vertex 0
@orientation(selector='bfs_children', directions=['below'])
@attribute(field='distance')
class BFSVertex:
    def __init__(self, value, distance=float('inf'), bfs_children=None):
        self.value = value
        self.distance = distance
        self.bfs_children = bfs_children or []
    
    def __repr__(self):
        return f"BFS({self.value}, d={self.distance})"

# Create BFS tree rooted at vertex 0
bfs_root = BFSVertex(0, 0)
bfs_level1 = [BFSVertex(1, 1), BFSVertex(2, 1)]
bfs_level2 = [BFSVertex(3, 2)]
bfs_level3 = [BFSVertex(4, 3)]

bfs_root.bfs_children = bfs_level1
bfs_level1[0].bfs_children = bfs_level2  # vertex 1 leads to vertex 3
bfs_level2[0].bfs_children = bfs_level3  # vertex 3 leads to vertex 4

print("BFS tree from vertex 0:")
print("Shows shortest path distances from source")
diagram(bfs_root)

BFS tree from vertex 0:
Shows shortest path distances from source


## 12. B-Trees (Optional)

**Relationalization**: Nodes with ordered keys; `childAt(node, i, child)` relations connecting parents to children at specific positions.

**Visualization**: Parent above children; keys grouped within nodes; multi-way branching clearly visible.

**What to notice**: The multi-way nature of B-trees becomes apparent — each node can hold multiple keys and have multiple children, optimized for disk access patterns.

In [25]:
# B-Tree node (simplified)
@orientation(selector='children', directions=['below'])
@group(field='keys', groupOn=0, addToGroup=1)
class BTreeNode:
    def __init__(self, keys=None, children=None, is_leaf=True):
        self.keys = keys or []
        self.children = children or []
        self.is_leaf = is_leaf
    
    def __repr__(self):
        return f"BNode({self.keys})"

print("B-Tree node class defined.")

B-Tree node class defined.


In [26]:
# B-Tree demonstration (degree 3)
# Root node with 2 keys
root = BTreeNode(keys=[10, 20], is_leaf=False)

# Child nodes
left_child = BTreeNode(keys=[5, 8])
middle_child = BTreeNode(keys=[12, 15])
right_child = BTreeNode(keys=[25, 30])

root.children = [left_child, middle_child, right_child]

# Apply grouping to show keys within nodes
for node in [root, left_child, middle_child, right_child]:
    annotate_group(node.keys, field='keys', groupOn=0, addToGroup=1)
    annotate_orientation(node.keys, selector='items', directions=['horizontal'])

print("B-Tree (degree 3) example:")
print("Root: [10, 20] with three children")
print("Left: [5, 8], Middle: [12, 15], Right: [25, 30]")
diagram(root)

B-Tree (degree 3) example:
Root: [10, 20] with three children
Left: [5, 8], Middle: [12, 15], Right: [25, 30]


## 13. Appendix: sPyTial Cheat-Sheet

Quick reference for common sPyTial decorators and functions used throughout this notebook:

In [27]:
# sPyTial Cheat Sheet - Common Patterns

print("=== CLASS DECORATORS ===")
print("@orientation(selector='field', directions=['left', 'right', 'above', 'below'])")
print("@group(field='items', groupOn=0, addToGroup=1)")
print("@atomColor(selector='self', value='red')")
print("@attribute(field='key')")
print("@cyclic(selector='root', direction='clockwise')")
print()

print("=== OBJECT ANNOTATION FUNCTIONS ===")
print("annotate_orientation(obj, selector='items', directions=['horizontal'])")
print("annotate_group(obj, field='elements', groupOn=0, addToGroup=1)")
print("annotate_atomColor(obj_list, selector='self', value='green')")
print("annotate_attribute(obj, selector='field', name='display_name')")
print("annotate_cyclic(obj_list, selector='root', direction='clockwise')")
print()

print("=== COMMON DIRECTION PATTERNS ===")
print("• Arrays/Lists: ['horizontal'] - left to right flow")
print("• Stacks: ['vertical'] - top to bottom")
print("• Trees: left=['below', 'left'], right=['below', 'right']")
print("• Linked lists: next=['right'] - pointer flow")
print("• Doubly linked: next=['right'], prev=['left']")
print("• Parent-child: ['above'] - hierarchical")
print()

print("=== VISUALIZATION TIPS ===")
print("• Use colors to highlight special nodes (roots, head/tail)")
print("• Group related elements (array items, hash buckets)")
print("• Show key attributes with @attribute decorators")
print("• Apply consistent spatial metaphors across similar structures")
print("• Keep examples small (≤10 nodes) for clarity")

# Demo tiny example
demo_list = [1, 2, 3]
annotate_orientation(demo_list, selector='items', directions=['horizontal'])
annotate_atomColor([demo_list[0]], selector='self', value='green')  # highlight first

print("\nTiny demo - horizontal list with green first element:")
diagram(demo_list)

=== CLASS DECORATORS ===
@orientation(selector='field', directions=['left', 'right', 'above', 'below'])
@group(field='items', groupOn=0, addToGroup=1)
@atomColor(selector='self', value='red')
@attribute(field='key')
@cyclic(selector='root', direction='clockwise')

=== OBJECT ANNOTATION FUNCTIONS ===
annotate_orientation(obj, selector='items', directions=['horizontal'])
annotate_group(obj, field='elements', groupOn=0, addToGroup=1)
annotate_atomColor(obj_list, selector='self', value='green')
annotate_attribute(obj, selector='field', name='display_name')
annotate_cyclic(obj_list, selector='root', direction='clockwise')

=== COMMON DIRECTION PATTERNS ===
• Arrays/Lists: ['horizontal'] - left to right flow
• Stacks: ['vertical'] - top to bottom
• Trees: left=['below', 'left'], right=['below', 'right']
• Linked lists: next=['right'] - pointer flow
• Doubly linked: next=['right'], prev=['left']
• Parent-child: ['above'] - hierarchical

=== VISUALIZATION TIPS ===
• Use colors to highlight spe

---

## Conclusion

This notebook has demonstrated how sPyTial makes classic CLRS data structures immediately visible through spatial visualization. By transforming structures into atoms and relations, then applying meaningful layout constraints, we can:

- **See structure clearly** — trees look like trees, lists flow linearly, graphs show connectivity
- **Verify properties visually** — heap property, BST ordering, hash distribution
- **Understand operations** — insertions, deletions, and traversals become spatial transformations
- **Debug more effectively** — structural problems become visual problems

The key insight is that **spatial representation reveals structural relationships** that are implicit in code but explicit in diagrams. With minimal annotation, sPyTial transforms any Python data structure into a meaningful visual representation.

For more examples and advanced usage, see the other notebooks in the `demos/` directory.